In [ ]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr
eigval = []
# Setup other constants
symbols  = ['N', 'H', 'H', 'H']

    # Define the molecule
for d in distances:
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
    def gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, shots=None, max_iter=100):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        # Map excitations to the wires the UCCSD circuit will act on
        s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

        wires=range(qubits)
        params = np.zeros(len(singles) + len(doubles))
        # Define the device
        dev = qml.device("lightning.qubit", wires=qubits, shots=shots)
        
        # Define the qnode
        if shots==None:
            @qml.qnode(dev, interface="autograd", diff_method="adjoint")
            def circuit(params, wires, s_wires, d_wires, hf_state):
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
        else:
            @qml.qnode(dev, interface="autograd")
            def circuit(params, wires, s_wires, d_wires, hf_state):
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)

        optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
        for n in range(max_iter):
            params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                    s_wires=s_wires, d_wires=d_wires,
                                                    hf_state=hf_state)

        gr = circuit(params, wires, s_wires, d_wires, hf_state)
        print('Ground state energy: ', gr)
        return params
    
    def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        # Map excitations to the wires the UCCSD circuit will act on
        s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        values =[]
        for t in range(1):
            if shots==0:
                #dev = qml.device("lightning.qubit", wires=qubits)
                dev = qml.device("lightning.qubit", wires=qubits)
            else:
                #dev = qml.device("default.qubit", wires=qubits,shots=shots)
                dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            #circuit for diagonal part
            @qml.qnode(dev)
            def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
                for w in occ:
                    qml.X(wires=w)
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
            #circuit for off-diagonal part
            @qml.qnode(dev)
            def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
                for w in occ1:
                    qml.X(wires=w)
                first=-1
                for v in occ2:
                    if v not in occ1:
                        if first==-1:
                            first=v
                            qml.Hadamard(wires=v)
                        else:
                            qml.CNOT(wires=[first,v])
                for v in occ1:
                    if v not in occ2:
                        if first==-1:
                            first=v
                            qml.Hadamard(wires=v)
                        else:
                            qml.CNOT(wires=[first,v])
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
            #final M matrix
            M = np.zeros((len(list1),len(list1)))
            for i in range(len(list1)):
                for j in range(len(list1)):
                    if i == j:
                        M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
            #print("diagonal parts done")
            for i in range(len(list1)):
                for j in range(len(list1)):
                    if i!=j:
                        Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                        M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
            #print("off diagonal terms done")
            #ERROR:not subtracting the gs energy
            eig,evec=np.linalg.eig(M)
            values.append(np.sort(eig))
        return values
    print(f"Computing for bond length: {d/r:.2f} Å")
    params = gs_exact(symbols,geometry, active_electrons, active_orbitals, charge,max_iter=500)

    eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge,params)
    eigval.append(eig)
    print('exact eigenvalues:', eigval)


In [ ]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr
eigval = []
# Setup other constants
symbols  = ['N', 'H', 'H', 'H']

    # Define the molecule
for d in distances:
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
    def gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, shots=None, max_iter=100):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        # Map excitations to the wires the UCCSD circuit will act on
        s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

        wires=range(qubits)
        params = np.zeros(len(singles) + len(doubles))
        # Define the device
        dev = qml.device("lightning.qubit", wires=qubits, shots=shots)
        
        # Define the qnode
        if shots==None:
            @qml.qnode(dev, interface="autograd", diff_method="adjoint")
            def circuit(params, wires, s_wires, d_wires, hf_state):
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
        else:
            @qml.qnode(dev, interface="autograd")
            def circuit(params, wires, s_wires, d_wires, hf_state):
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)

        optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
        for n in range(max_iter):
            params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                    s_wires=s_wires, d_wires=d_wires,
                                                    hf_state=hf_state)

        gr = circuit(params, wires, s_wires, d_wires, hf_state)
        print('Ground state energy: ', gr)
        return params
    
    def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        # Map excitations to the wires the UCCSD circuit will act on
        s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        values =[]
        for t in range(1):
            if shots==0:
                #dev = qml.device("lightning.qubit", wires=qubits)
                dev = qml.device("lightning.qubit", wires=qubits)
            else:
                #dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
                dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            #circuit for diagonal part
            @qml.qnode(dev)
            def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
                for w in occ:
                    qml.X(wires=w)
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
            #circuit for off-diagonal part
            @qml.qnode(dev)
            def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
                for w in occ1:
                    qml.X(wires=w)
                first=-1
                for v in occ2:
                    if v not in occ1:
                        if first==-1:
                            first=v
                            qml.Hadamard(wires=v)
                        else:
                            qml.CNOT(wires=[first,v])
                for v in occ1:
                    if v not in occ2:
                        if first==-1:
                            first=v
                            qml.Hadamard(wires=v)
                        else:
                            qml.CNOT(wires=[first,v])
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
            #final M matrix
            M = np.zeros((len(list1),len(list1)))
            for i in range(len(list1)):
                for j in range(len(list1)):
                    if i == j:
                        M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
            #print("diagonal parts done")
            for i in range(len(list1)):
                for j in range(len(list1)):
                    if i!=j:
                        Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                        M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
            #print("off diagonal terms done")
            #ERROR:not subtracting the gs energy
            eig,evec=np.linalg.eig(M)
            values.append(np.sort(eig))
        return values
    print(f"Computing for bond length: {d/r:.2f} Å")
    params = gs_exact(symbols,geometry, active_electrons, active_orbitals, charge,max_iter=500)

    eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge,params,shots=1000)
    eigval.append(eig)
    print('1k eigenvalues:', eigval)


In [ ]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr
eigval = []
# Setup other constants
symbols  = ['N', 'H', 'H', 'H']

    # Define the molecule
for d in distances:
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
    def gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, shots=None, max_iter=100):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        # Map excitations to the wires the UCCSD circuit will act on
        s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

        wires=range(qubits)
        params = np.zeros(len(singles) + len(doubles))
        # Define the device
        dev = qml.device("lightning.qubit", wires=qubits, shots=shots)
        
        # Define the qnode
        if shots==None:
            @qml.qnode(dev, interface="autograd", diff_method="adjoint")
            def circuit(params, wires, s_wires, d_wires, hf_state):
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
        else:
            @qml.qnode(dev, interface="autograd")
            def circuit(params, wires, s_wires, d_wires, hf_state):
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)

        optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
        for n in range(max_iter):
            params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                    s_wires=s_wires, d_wires=d_wires,
                                                    hf_state=hf_state)

        gr = circuit(params, wires, s_wires, d_wires, hf_state)
        print('Ground state energy: ', gr)
        return params
    
    def qsceom(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        # Map excitations to the wires the UCCSD circuit will act on
        s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        values =[]
        for t in range(1):
            if shots==0:
                #dev = qml.device("lightning.qubit", wires=qubits)
                dev = qml.device("lightning.qubit", wires=qubits)
            else:
                #dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
                dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            #circuit for diagonal part
            @qml.qnode(dev)
            def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
                for w in occ:
                    qml.X(wires=w)
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
            #circuit for off-diagonal part
            @qml.qnode(dev)
            def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
                for w in occ1:
                    qml.X(wires=w)
                first=-1
                for v in occ2:
                    if v not in occ1:
                        if first==-1:
                            first=v
                            qml.Hadamard(wires=v)
                        else:
                            qml.CNOT(wires=[first,v])
                for v in occ1:
                    if v not in occ2:
                        if first==-1:
                            first=v
                            qml.Hadamard(wires=v)
                        else:
                            qml.CNOT(wires=[first,v])
                qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
                return qml.expval(H)
            #final M matrix
            M = np.zeros((len(list1),len(list1)))
            for i in range(len(list1)):
                for j in range(len(list1)):
                    if i == j:
                        M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
            #print("diagonal parts done")
            for i in range(len(list1)):
                for j in range(len(list1)):
                    if i!=j:
                        Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                        M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
            #print("off diagonal terms done")
            #ERROR:not subtracting the gs energy
            eig,evec=np.linalg.eig(M)
            values.append(np.sort(eig))
        return values
    print(f"Computing for bond length: {d/r:.2f} Å")
    params = gs_exact(symbols,geometry, active_electrons, active_orbitals, charge,max_iter=500)

    eig = qsceom(symbols, geometry, active_electrons, active_orbitals, charge,params,shots=10000)
    eigval.append(eig)
    print('10k eigenvalues:', eigval)
